In [ ]:
import numpy as np
import scipy.stats as sstats
from matplotlib import pyplot as plt

In [ ]:
plt.xkcd()

In [ ]:
import cbayes.sample as samp
import cbayes.distributions as dist
import cbayes.solve as solve

In [ ]:
def QoI_fun(lam):
    return np.power(lam, 5).reshape(-1, 1)

## Consistent Bayes

In [ ]:
N = 50000
input_dim = 1
output_dim = input_dim
s_set = samp.sample_set(size=(N, input_dim))
for i in range(input_dim):# define prior distribution
    s_set.set_dist('uni', {'loc':-1, 'scale':2}, 0) # uniform
#     s_set.set_dist('norm', {'loc':0, 'scale':0.5}, 0) # normal distribution 
s_set.generate_samples()
p_set = samp.map_samples_and_create_problem(s_set, QoI_fun)

In [ ]:
plt.scatter(p_set.input.samples[:], p_set.output.samples[:])
plt.show()

In [ ]:
sd = 0.1
lo = 0.25
p_set.set_observed_dist('norm', dim=0, kwds={'loc':lo, 'scale': sd})
# FIX THE BUG WHERE YOU CANNOT OVERWRITE THIS... (didn't I already do this?... maybe just forgot to apply it to the observed dist part)

In [ ]:
# p_set.compute_pushforward_dist(method='sk',kwds={'kernel': 'gaussian', 'bandwidth':0.1})

In [ ]:
p_set.compute_pushforward_dist(method='scipy')

In [ ]:
pf = p_set.pushforward_dist

In [ ]:
x = np.linspace(-1.0,1.0,1000)
plt.plot(x,pf.pdf(x))

In [ ]:
p_set.set_ratio()

In [ ]:
solve.problem(p_set)

In [ ]:
plt.scatter(p_set.input.samples, p_set.ratio)
plt.show()

## Statistical Bayes

In [ ]:
D = dist.parametric_dist(output_dim)
for i in range(output_dim):
    D.set_dist(i, 'norm',{'loc':lo, 'scale':sd })


In [ ]:
likelihood = D.pdf(p_set.output.samples)
prior = p_set.input.dist.pdf(p_set.input.samples)

In [ ]:
post = likelihood*prior

In [ ]:
plt.scatter(p_set.input.samples, post)
plt.show()

In [ ]:
# PS = post/np.max(post)
# PC = p_set.ratio/np.max(p_set.ratio)
# ps = np.concatenate([s_set.samples, PS.reshape(-1,1)], axis=1)
# pc = np.concatenate([s_set.samples, PC.reshape(-1,1)], axis=1)

In [ ]:
indS = solve.perform_accept_reject(s_set.samples, post)
indC = solve.perform_accept_reject(s_set.samples, p_set.ratio)

In [ ]:
cb_ps_den = dist.gkde(p_set.input.samples[indC])
sb_ps_den = dist.gkde(p_set.input.samples[indS])
cb_pf_den = dist.gkde(p_set.output.samples[indC])
sb_pf_den = dist.gkde(p_set.output.samples[indS])

In [ ]:
plt.plot(x, cb_pf_den.pdf(x), c='b', label='CB sol')
plt.plot(x, sb_pf_den.pdf(x), c='k', label='SB sol')
plt.show()

In [ ]:
plt.plot(x, cb_ps_den.pdf(x), c='b', label='CB sol')
plt.plot(x, sb_ps_den.pdf(x), c='k', label='SB sol')
plt.show()